In [2]:
import numpy as np

In [3]:
import keras
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt",shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [5]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [7]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [8]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1

In [9]:
import tensorflow as tf
train_size = dataset_size*90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [10]:
n_steps = 100
window_length = n_steps+1
dataset = dataset.window(window_length,shift=1,drop_remainder=True)

In [11]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [12]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows:(windows[:,:-1],windows[:,1:]))

In [13]:
dataset = dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch)
)
dataset = dataset.prefetch(1)

In [14]:
model = keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam")
history = model.fit(dataset,epochs=20)

Epoch 1/20
   1066/Unknown 180s 166ms/step - loss: 2.4756

In [15]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X,max_id)

In [16]:
X_new = preprocess(["how are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
def next_char(text,temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0,-1:,:]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits,num_samples=1)+1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [17]:
def complete_text(text,nchars=50,temperature=1):
  for _ in range(n_chars):
    text += next_char(text,temperature)
  return text

In [18]:
print(complete_text("t",temperature=0.2))

NameError: name 'n_chars' is not defined

In [ ]:
print(complete_text("w",temperature=1))

In [19]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length,shift=n_steps,drop_remainder=True)
dataset = dataset.flat_map(lambda window:window.batch(window_length))
dataset = dataset.batch(1)

dataset = dataset.map(lambda X_Y: (tf.one_hot(X_Y[0], depth=max_id), X_Y[1]))

dataset = dataset.prefetch(1)

In [20]:
model = keras.models.Sequential([
    keras.layers.Input(batch_shape=(batch_size, None, max_id)),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation="softmax"))
])


In [21]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.reset_states()

In [22]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam")

In [ ]:
model.fit(dataset,epochs=50,callbacks=[ResetStatesCallback])

In [23]:
(X_train,y_train),(X_test,y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [24]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_+3:word for word,id_ in word_index.items()}
for id_,token in enumerate(("<pad>","<sos>","<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [25]:
import tensorflow_datasets as tfds
datasets,info = tfds.load("imdb_reviews",as_supervised=True,with_info=True)
train_size = info.splits["train"].num_examples

In [26]:
def preprocess(X_batch,y_batch):
    X_batch = tf.strings.substr(X_batch,0,300)
    X_batch = tf.strings.regex_replace(X_batch,b"<br\\s*/?>",b" ")
    X_batch = tf.strings.regex_replace(X_batch,b"[^a-zA-Z']",b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"),y_batch

In [27]:
from collections import Counter
vocabulary = Counter()
for X_batch,y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

2025-07-30 14:02:25.200087: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [28]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [29]:
vocab_size = 1000
truncated_vocabulary = [
    word for word,count in vocabulary.most_common()[:vocab_size]
]

In [30]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary),dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words,word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)

In [31]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[  22,   12,   11, 1053]])>

In [32]:
def encode_words(X_batch,y_batch):
    return table.lookup(X_batch),y_batch
train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [33]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size+num_oov_buckets,embed_size,input_shape=[None]),
    keras.layers.GRU(128,return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [34]:
history = model.fit(train_set,epochs=5)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 86ms/step - accuracy: 0.5069 - loss: 0.6940
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 90ms/step - accuracy: 0.6173 - loss: 0.6341
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 87ms/step - accuracy: 0.7700 - loss: 0.4805
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 86ms/step - accuracy: 0.7997 - loss: 0.4361
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 86ms/step - accuracy: 0.8218 - loss: 0.4009


In [37]:
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(
    lambda inputs: K.not_equal(inputs, 0),
    output_shape=inputs.shape
)(inputs)
z = keras.layers.Embedding(vocab_size+num_oov_buckets,embed_size)(inputs)
z = keras.layers.GRU(128,return_sequences=True)(z,mask=mask)
z = keras.layers.GRU(128)(z,mask=mask)
outputs = keras.layers.Dense(1,activation="sigmoid")(z)
model = keras.Model(inputs=[inputs],outputs=[outputs])

In [40]:
import tensorflow_hub as hub
model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                         dtype=tf.string,input_shape=[],output_shape=[50]),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="adam",
              metrics=["accuracy"])




Failed to import tf_keras. Please note that tf_keras is not installed by default when you install tensorflow_hub. This is so that users can decide which tf_keras package to use. To use tensorflow_hub, please install a current version of tf_keras.




ImportError: cannot import name 'check_pinned' from 'tensorflow.python.ops.gen_experimental_dataset_ops' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/ops/gen_experimental_dataset_ops.py)

In [ ]:
datasets,info = tfds.load("imdb_reviews",as_supervised=True,with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set,epochs =5)